In [68]:
from rdflib import *
import json
import re
ns = "https://buffalo.edu/lfocourse/assignment1p2/"


def get_resource(local):
    label = re.sub("\W+","",local)    
    return URIRef(f'{ns}{label}') , Literal(local, lang='en')

constraint_set = {}
for p in dir(OWL):
    if p.__str__().find("operty") >=0 : 
        f=re.findall("#(.+)Property",p.__str__())
        if len(f) > 0:
            constraint_set[f[0]] = p
##print(json.dumps(constraint_set, indent=2))

def get_prop(selector):
    ptype = constraint_set.get(selector)
    res, lab = get_resource(selector.lower() + " test prop")
    return res,lab, ptype

def subproperty(constraint_1, constraint_2):
    text = f"Suppose B owl:subPropertyOf A, A is {constraint_1} and B is {constraint_2}."
    return f"<p>{text}</p>"

def inverseof(constraint_1, constraint_2):
    text =  f"Suppose B owl:inverseOf A, A is {constraint_1} and B is {constraint_2}."
    return f"<p>{text}</p>"
def owl_fn(constraint_1, constraint_2,ipl):
    return f"{constraint_1}_{ipl}_{constraint_2}.ttl"

def get_table(constraint_1, constraint_2): 
    df = "OK"
    grid = []
    em = "_"
    #grid.append(["",constraint_2,"","",""])
    grid.append(["","A","Ai","B","Bi"])
    grid.append(["A" ,em,em,em,em])
    grid.append(["Ai",df,em,em,em])
    grid.append(["B", df,df,em,em])
    grid.append(["Bi",df,df,df,em]) 
    ogrid= []
    for g in grid:
        ogrid.append("<tr>" + "".join([f"<td>{x}</td>" for x in g]) + "</tr>") 
    #return ogrid    
    return "<table>\n" + "\n".join(ogrid) + "\n</table>"


def get_model(constraint_1, constraint_2, inter_prop, inter_individual_prop=None):
    #comment = f"""This model is a test to examine the consistency behaviours 
    #    of two properties A and B where A is {constraint_1} and B
    #    is  {constraint_2} and {constraint_1} is {inter_prop} of {constraint_2}."""
    #if inter_individual_prop is not None:
    #    comment += f"""Furthermore, we set pairwise {inter_individual_prop} between 
    #        our test instances (x,y,z) to analyze any effects on the overall model
    #        consistency.  """
    model = Graph()
    model.bind("dev",ns)
    
    #model.add((ns,RDF.type,OWL.Ontology))
    #model.add((ns, RDFS.comment, Literal(comment, lang = 'en')))
    

    Ao, Al , At = get_prop(constraint_1) 
    Bo, Bl , Bt = get_prop(constraint_2) 
    model.add((Ao,RDF.type, OWL.ObjectProperty))
    model.add((Ao,RDF.type, At))
    model.add((Ao,RDFS.label,Al)) 
    model.add((Bo,RDF.type, OWL.ObjectProperty))    
    model.add((Bo,RDF.type, Bt))
    model.add((Bo,RDFS.label,Bl)) 
    model.add((Bo,inter_prop,Ao))
    
    o, ol = get_resource("Test Thing")
    model.add((o,RDF.type,OWL.Class))
    model.add((o,RDFS.label,ol))
    
    x, xl = get_resource('x') 
    model.add((x,RDF.type,o))
    model.add((x,RDFS.label, xl))
    y, yl = get_resource('y') 
    model.add((y,RDF.type,o))
    model.add((y,RDFS.label, yl))
    z, zl = get_resource('z')     
    model.add((z,RDF.type,o))
    model.add((z,RDFS.label, zl))
    
    ## assertions between individuals pairwise
    if inter_individual_prop is not None:
        individuals = [x,y,z]
        for i in range(len(individuals)):
            for j in range(len(individuals)):
                if i != j : 
                    model.add((individuals[i],inter_individual_prop ,individuals[j]))
                        
    return model

In [73]:
from datetime import datetime
print(datetime.now())
F="Functional"
IF = "InverseFunctional"
T = "Transitive"
S = "Symmetric"
AS = "Asymmetric"
R = "Reflexive"
IR = "Irreflexive"
combinations = """
F,IF,S,AS,R,IR
IF,S,AS,R,IR
T,S,R
S,R,IR
AS,IR"""
C= [ c.split(",") for c in combinations.split("\n") if len(c)>0]

readme = []
## no_interindividual_assertion
  
prop_configs = {
    'no_interindividual_assertion': None, 
    'all_same_individuals': OWL.sameAs ,
    'all_different_individuals': OWL.differentFrom
    
}

for dr in prop_configs:
    iiprop = prop_configs[dr]
    print(dr,iiprop )
    
           
    if True:
        for c in C:
            for i in range(1,len(c)):
                c1 = eval(c[0])
                c2 = eval(c[i])
                title = f"### Properties A,B where A is {c1} and B is and {c2}"
                table = get_table(c1,c2)


                inv_model_a_b = get_model(c1,c2,OWL.inverseOf,iiprop)
                inv_stmt_a_b = inverseof(c1,c2)
                inv_owl_a_b = owl_fn(c1,c2,"inverseOf")
                
                inv_model_b_a = get_model(c2,c1,OWL.inverseOf,iiprop)
                inv_stmt_b_a = inverseof(c2,c1)
                inv_owl_b_a = owl_fn(c2,c1,"inverseOf")                

                subprop_model_a_b = get_model(c1,c2,RDFS.subPropertyOf,iiprop)
                subprop_stmt_a_b = subproperty(c1,c2)
                subprop_owl_a_b = owl_fn(c1,c2,"subPropertyOf")

                subprop_model_b_a = get_model(c2,c1,RDFS.subPropertyOf,iiprop)            
                subprop_stmt_b_a = subproperty(c2,c1) 
                subprop_owl_b_a = owl_fn(c2,c1,"subPropertyOf")

                ## do we need to study this in both directions? 
                ##subprop_model_b_a = get_model(c1,c2,RDFS.subPropertyOf)            

                readme.append(title)
                readme.append("\n")
                readme.append(table)
                readme.append("\n")
                readme.append(subprop_stmt_a_b) 
                readme.append(f'<a href="{subprop_owl_a_b}">{subprop_owl_a_b}</a>')  
                readme.append("\n")
                readme.append(subprop_stmt_b_a) 
                readme.append(f'<a href="{subprop_owl_b_a}">{subprop_owl_b_a}</a>')             
                readme.append("\n")
                readme.append(inv_stmt_a_b) 
                readme.append(f'<a href="{inv_owl_a_b}">{inv_owl_a_b}</a>')            
                readme.append("\n")
                readme.append(inv_stmt_b_a) 
                readme.append(f'<a href="{inv_owl_b_a}">{inv_owl_b_a}</a>')            
                readme.append("\n")                
                readme.append("\n")  
                open(f"{dr}/{subprop_owl_a_b}",'w').write(subprop_model_a_b.serialize(format = 'ttl'))            
                open(f"{dr}/{subprop_owl_b_a}",'w').write(subprop_model_b_a.serialize(format = 'ttl'))                        
                open(f"{dr}/{inv_owl_a_b}",'w').write(inv_model_a_b.serialize(format = 'ttl'))

    ##print(get_model("Asymmetric","Reflexive",OWL.inverseOf).serialize(format='ttl') ) 
    if True:       
        open("project1_part2.md",'w').write("".join(readme)) 


2024-09-26 20:48:02.439114
no_interindividual_assertion None
all_same_individuals http://www.w3.org/2002/07/owl#sameAs
all_different_individuals http://www.w3.org/2002/07/owl#differentFrom


In [64]:
[p for p in dir(OWL) if 'different' in str(p)]

[rdflib.term.URIRef('http://www.w3.org/2002/07/owl#differentFrom')]

In [58]:
A="""Asymmetric_subPropertyOf_Irreflexive.ttl
Functional_inverseOf_Asymmetric.ttl
Functional_inverseOf_InverseFunctional.ttl
Functional_inverseOf_Irreflexive.ttl
Functional_inverseOf_Reflexive.ttl
Functional_inverseOf_Symmetric.ttl
Functional_subPropertyOf_Asymmetric.ttl
Functional_subPropertyOf_InverseFunctional.ttl
Functional_subPropertyOf_Irreflexive.ttl
Functional_subPropertyOf_Reflexive.ttl
Functional_subPropertyOf_Symmetric.ttl
InverseFunctional_inverseOf_Asymmetric.ttl
InverseFunctional_inverseOf_Irreflexive.ttl
InverseFunctional_inverseOf_Reflexive.ttl
InverseFunctional_inverseOf_Symmetric.ttl
InverseFunctional_subPropertyOf_Asymmetric.ttl
InverseFunctional_subPropertyOf_Irreflexive.ttl
InverseFunctional_subPropertyOf_Reflexive.ttl
InverseFunctional_subPropertyOf_Symmetric.ttl
Symmetric_inverseOf_Irreflexive.ttl
Symmetric_inverseOf_Reflexive.ttl
Symmetric_subPropertyOf_Irreflexive.ttl
Symmetric_subPropertyOf_Reflexive.ttl
Transitive_inverseOf_Reflexive.ttl
Transitive_inverseOf_Symmetric.ttl
Transitive_subPropertyOf_Reflexive.ttl""".split("\n")
print(" ".join(A)) 


Asymmetric_subPropertyOf_Irreflexive.ttl Functional_inverseOf_Asymmetric.ttl Functional_inverseOf_InverseFunctional.ttl Functional_inverseOf_Irreflexive.ttl Functional_inverseOf_Reflexive.ttl Functional_inverseOf_Symmetric.ttl Functional_subPropertyOf_Asymmetric.ttl Functional_subPropertyOf_InverseFunctional.ttl Functional_subPropertyOf_Irreflexive.ttl Functional_subPropertyOf_Reflexive.ttl Functional_subPropertyOf_Symmetric.ttl InverseFunctional_inverseOf_Asymmetric.ttl InverseFunctional_inverseOf_Irreflexive.ttl InverseFunctional_inverseOf_Reflexive.ttl InverseFunctional_inverseOf_Symmetric.ttl InverseFunctional_subPropertyOf_Asymmetric.ttl InverseFunctional_subPropertyOf_Irreflexive.ttl InverseFunctional_subPropertyOf_Reflexive.ttl InverseFunctional_subPropertyOf_Symmetric.ttl Symmetric_inverseOf_Irreflexive.ttl Symmetric_inverseOf_Reflexive.ttl Symmetric_subPropertyOf_Irreflexive.ttl Symmetric_subPropertyOf_Reflexive.ttl Transitive_inverseOf_Reflexive.ttl Transitive_inverseOf_Symme

In [61]:
!ls open('no_interindividual_assertion_logs/pellet_InverseFunctional_inverseOf_Asymmetric.txt','r').read()

zsh: parse error near `()'
